In [1]:
%pylab inline
import numpy as np
import matplotlib.pyplot as plt 
data = sc.textFile('data.txt')

Populating the interactive namespace from numpy and matplotlib


In [2]:
words = data.flatMap(lambda x: x.split()).filter(lambda x: x != x.upper())

def is_number(x): #from stackoverflow...
    try:
        int(x)
        return False
    except ValueError:
        return True

words = words.filter(is_number)

In [3]:
string = words.collect()

In [4]:
trips = []
for i in range(len(string)-2):
    trips.append(((string[i],string[i+1]),(string[i+2])))

In [5]:
trips = sc.parallelize(trips)

In [6]:
def count(l):
    return [(x,l.count(x)) for x in set(l)] #stackoverflow for determining count of words occurances in a list

In [7]:
RDD = trips.groupByKey().map(lambda x: (x[0],list(x[1])))

In [50]:
Shakespeare = RDD.mapValues(lambda x: count(x))

In [51]:
Shakespeare.take(10)

[((u'smell', u'onions;'), [(u'shall', 1)]),
 ((u'meantime,', u'lady,'), [(u"I'll", 1)]),
 ((u'a', u'master?'), [(u'Bear', 1)]),
 ((u'some', u'while'), [(u'a', 1)]),
 ((u'Priam', u'is'), [(u'in', 1)]),
 ((u'East,', u'Say'), [(u'thou,', 1)]),
 ((u'tall;', u'yet'), [(u'for', 1)]),
 ((u'Should', u'hold'), [(u'up', 1), (u'her', 1)]),
 ((u'until', u'meet'), [(u'thee', 1)]),
 ((u'and', u"think'st"), [(u'it', 1)])]

In [257]:
test = Shakespeare.filter(lambda x: x[0]== (u'Now', u'is'))
test.take(20)

[((u'Now', u'is'),
  [(u'a', 1),
   (u'be', 1),
   (u'his', 1),
   (u'that', 1),
   (u'this', 1),
   (u'it', 3),
   (u'Mortimer', 1),
   (u'the', 9),
   (u'my', 2),
   (u'your', 1),
   (u'he', 1)])]

In [235]:
def bias(x):
    bias_array = []
    for i in x:
        j = i[1]
        for k in range(j):
            bias_array.append(i[0])
    #print bias_array        
    return random.choice(bias_array)
    

In [243]:
def markov(words):
    prediction = []
    options = range(len(Shakespeare.collect()))           
    start_index = random.choice(options)
    
    starting_pair = Shakespeare.collect()[start_index][0]
    
    prediction.append(str(starting_pair[0] ))
    prediction.append(str(starting_pair[1] ))
    i=2
    pair = (starting_pair[0],starting_pair[1])
    while i < (words):
        get_options = Shakespeare.lookup(pair)
        #print get_options[0]
        #print get_options[0][0]
        #print get_options[0][0][0]
        #print len(list(get_options[0]))
        next_word = bias(get_options[0])
        #print next_word
        prediction.append(str(next_word ))
        pair = (pair[1],next_word)
        #print pair
        i+=1
    return prediction


In [255]:
Generate_Shakespeare23 = markov(20)
print Generate_Shakespeare23
print " ".join(Generate_Shakespeare23)

['mind', 'he', 'bears,', 'And', 'his', 'great', 'office,', 'that', 'his', 'answer?', 'Ay,', 'my', 'good', 'lord.', 'My', 'lady,', 'to', 'yourself!', 'According', 'to']
mind he bears, And his great office, that his answer? Ay, my good lord. My lady, to yourself! According to


In [256]:
Generate_Shakespeare = []
for i in range(10):
    Generate_Shakespeare.append(markov(20))

KeyboardInterrupt: 

In [78]:
def bias(x):
    bias_array = []
    for i in x:
        j = i[1]
        for k in range(j):
            bias_array.append(i[0])
    #print bias_array        
    return random.choice(bias_array)
    

In [234]:
def markov(words):
    prediction = []
    options = range(len(test23.collect()))           
    start_index = random.choice(options)
    
    starting_pair = test23.collect()[start_index][0]
    
    prediction.append(str(starting_pair[0] ))
    prediction.append(str(starting_pair[1] ))
    i=2
    pair = (starting_pair[0],starting_pair[1])
    while i < (words):
        get_options = test23.lookup(pair)
        #print get_options[0]
        #print get_options[0][0]
        #print get_options[0][0][0]
        #print len(list(get_options[0]))
        next_word = bias(get_options[0])
        print next_word
        prediction.append(str(next_word ))
        pair = (pair[1],next_word)
        print pair
        i+=1
    return prediction
markov(3)

you
('are', 'you')


['how', 'are', 'you']

In [211]:
test23 = [[("how", "are"),[("you",4),("the",3)]],[("are","the"),[("people",1),("children",3)]],[("the","people"),[("doing",2),("living",2)]]]

In [212]:
test23 = sc.parallelize(test23)

In [213]:
test23.collect()[2]

[('the', 'people'), [('doing', 2), ('living', 2)]]